In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import csv
import pandas as pd
import re

In [2]:
# time
t1 = time.time()
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t1))


'2019-05-18 10:44:34'

In [3]:
bidInfo = {}

In [4]:
csvFile = open("namelist.csv","w+")
writer = csv.writer(csvFile)

bidInfo = {}

header = ("name","date","title","consultPrice","currentPrice","bidCount","delayCount","applyCount","area","url")

writer.writerow(header)

#writer.writerow(('name','url'))



83

In [5]:
sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2" \
                 "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-13&spm=a213w.3064813.9001.2"  



In [6]:
# Start the driver
driver = webdriver.Chrome()
# Setup wait for later
wait = WebDriverWait(driver, 10)

driver.get(sourceUrl)

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

In [7]:
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))
#pages = driver.find_element_by_class_name("page-total").text


<selenium.webdriver.remote.webelement.WebElement (session="b90762fe753cb51c7847b4ea3cc355cc", element="0.4031990617606025-1")>

In [8]:
#test page 
pages = 1

In [9]:
for page in range(1,int(pages)+1):
    if (page > 1):
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
    
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
    data = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')
    jsonData =  json.loads(data)
    items = jsonData["data"]
    
    itemLen = len(items)
    for item in range(itemLen):    
    #for item in range(4):        
        bidInfo["status"] = items[item]["status"]

        
        if(bidInfo["status"] == "done"):
            
            bidInfo["id"] = items[item]["id"]
            bidInfo["title"] = items[item]["title"]
            bidInfo["consultPrice"] = items[item]["consultPrice"]
            bidInfo["currentPrice"] = items[item]["currentPrice"]
            bidInfo["bidCount"] = items[item]["bidCount"]
            bidInfo["delayCount"] = items[item]["delayCount"]
            bidInfo["applyCount"] = items[item]["applyCount"]

            url = "#pai-item-"+str(bidInfo["id"])
            # Click the link which opens in a new window
            driver.find_element_by_css_selector(url).click()
            time.sleep(1) # have to wait sometime

            # Wait for the new window or tab
            wait.until(EC.number_of_windows_to_be(2))
            handles = driver.window_handles
            driver.switch_to.window(handles[1])
            # size
            
            urlNew = driver.current_url
            url2 = str(urlNew)+"#J_Confirmation"
            driver.get(url2)
            
            
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))
            info = driver.find_element_by_class_name("c-content").text
            index = info.find("通过")
            
            # getName
            bidInfo["name"] = info[4:index]
            # getDate
            bidInfo["date"] = driver.find_element_by_class_name("c-stamp").text
            

            #print
            print("Name: "+bidInfo["name"]+": Page :"+str(page)+"   Item :"+str(item))



           # get area
            driver.get(str(urlNew)+"#NoticeDetail")
            time.sleep(1)
            notice = driver.find_element_by_id("J_NoticeDetail").get_attribute('textContent')            
            left = notice.find("面积")
            right = notice.find("方米")
            size = notice[(left+2) : right]
            
            if(len(size) < 20):
                area = (re.findall(r"\d+\.?\d*",size))[0]
            else:
                area = 0
            
            bidInfo["area"] = area
            bidInfo["url"] = urlNew

            print("area :" + str(area) )



            
            # add to file
#            writer.writerow((bidInfo["name"],driver.current_url))
#           ("name","date","title","consultPrice","currentPrice","bidCount","delayCount","applyCount","area","url")

            writer.writerow((bidInfo["name"],bidInfo["date"],bidInfo["title"],
                                   bidInfo["consultPrice"],bidInfo["currentPrice"],
                                   bidInfo["bidCount"],bidInfo["delayCount"],bidInfo["applyCount"],
                                   bidInfo["area"],bidInfo["url"]
                                 ))
                                 

            driver.close()
            driver.switch_to.window(handles[0])
        else:
            print("***item failed******"+": Page :"+str(page)+"   Item :"+str(item))



***item failed******: Page :1   Item :0
***item failed******: Page :1   Item :1
***item failed******: Page :1   Item :2
***item failed******: Page :1   Item :3
Name: 杨宁: Page :1   Item :4
area :116.97
Name: 魏环利: Page :1   Item :5
area :85.05
Name: 李玉芹: Page :1   Item :6
area :41.79
Name: 于德龙: Page :1   Item :7
area :95.14
***item failed******: Page :1   Item :8
Name: 张丹琪: Page :1   Item :9
area :137.02
***item failed******: Page :1   Item :10
***item failed******: Page :1   Item :11
***item failed******: Page :1   Item :12
***item failed******: Page :1   Item :13
***item failed******: Page :1   Item :14
***item failed******: Page :1   Item :15
***item failed******: Page :1   Item :16
***item failed******: Page :1   Item :17
***item failed******: Page :1   Item :18
***item failed******: Page :1   Item :19
***item failed******: Page :1   Item :20
***item failed******: Page :1   Item :21
***item failed******: Page :1   Item :22
***item failed******: Page :1   Item :23
***item failed******

In [10]:
driver.quit()

In [11]:
csvFile.close()
csv = pd.read_csv('namelist.csv', encoding='utf-8')
csv.to_excel('namelist.xlsx', sheet_name='data')

In [12]:
t2 = time.time()
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t2))

seconds = t2 - t1
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ("%02d:%02d:%02d" % (h, m, s))


00:00:32


In [13]:
#            print("URL: "+driver.current_url)
#             print("id:"+str(bidInfo["id"])+" title: "+bidInfo["title"])
#             print("consultPrice:"+str(bidInfo["consultPrice"]) + " currentPrice:" +str(bidInfo["currentPrice"]))
#             print("bidCount:"+str(bidInfo["bidCount"]) +" delayCount:"+str(bidInfo["delayCount"])+" applyCount:"+str(bidInfo["applyCount"]))
#             print("date:"+bidInfo["date"])